In [351]:
from tqdm import tqdm_notebook
import pandas as pd
import seaborn as sns
import json
import matplotlib.pyplot as plt
import pickle5 as pickle
from glob import glob
from datetime import datetime
import re
import numpy as np
from matplotlib.colors import ListedColormap
from scipy.stats import mannwhitneyu
threshold={'hatespeech':0.9,'fearspeech':0.7,'normal':0.5}
dict_labels={'hatespeech':2,'fearspeech':1,'normal':0}
reverse_dict_labels={dict_labels[key]:key for key in dict_labels.keys()}

In [352]:
import re
import urllib.parse as urlparse
import emoji


GENERIC_TLDS = [
    'aero', 'asia', 'biz', 'com', 'coop', 'edu', 'gov', 'info', 'int', 'jobs', 
    'mil', 'mobi', 'museum', 'name', 'net', 'org', 'pro', 'tel', 'travel', 'cat'
    ]

def get_domain(url):
    hostname = urlparse.urlparse(url.lower()).netloc
    if hostname == '':
        # Force the recognition as a full URL
        hostname = urlparse.urlparse('http://' + url).netloc

    # Remove the 'user:passw', 'www.' and ':port' parts
    hostname = hostname.split('@')[-1].split(':')[0].lstrip('www.').split('.')

    num_parts = len(hostname)
    if (num_parts < 3) or (len(hostname[-1]) > 2):
        return '.'.join(hostname[:-1])
    if len(hostname[-2]) > 2 and hostname[-2] not in GENERIC_TLDS:
        return '.'.join(hostname[:-1])
    if num_parts >= 3:
        return '.'.join(hostname[:-2])


In [385]:
dict_urls={}

In [ ]:
files = sorted(glob('../../Gab_Data/new_features_old_gab/gab_fear_hate_features*.pickle'))
for file in tqdm_notebook(files,total=len(files)):
    with open(file, 'rb') as handle:
        Gab_keyword_match = pickle.load(handle)
        
    if('predicted_probab' not in Gab_keyword_match[1].keys()):
        continue
    
    for element in Gab_keyword_match:
        try:
            dt = datetime.fromisoformat(element['post_create_time'])
            key_new=str(dt.month)+'/'+str(dt.year)
        except KeyError:
            continue
            
        
        
        temp=element['post_body']
        all_urls = re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', temp)
        
        labels=[]

        for i in range(len(element['predicted_probab'])):
            if(element['predicted_probab'][i]>threshold[reverse_dict_labels[i]]):
                labels.append(reverse_dict_labels[i])
        
        for url in all_urls:
            if('fearspeech' in labels):
                try:
                    dict_urls[get_domain(url)][url]['fearspeech']+=1
                except KeyError:
                    try:
                        dict_urls[get_domain(url)][url]={}
                        dict_urls[get_domain(url)][url]['fearspeech']=1
                    except KeyError:
                        dict_urls[get_domain(url)]={}
                        dict_urls[get_domain(url)][url]={}
                        dict_urls[get_domain(url)][url]['fearspeech']=1

                            
                        
            elif('hatespeech' in labels):
                try:
                    dict_urls[get_domain(url)][url]['hatespeech']+=1
                except KeyError:
                    try:
                        dict_urls[get_domain(url)][url]={}
                        dict_urls[get_domain(url)][url]['hatespeech']=1
                    except KeyError:
                        dict_urls[get_domain(url)]={}
                        dict_urls[get_domain(url)][url]={}
                        dict_urls[get_domain(url)][url]['hatespeech']=1
            
            


            else:
                pass


<ipython-input-386-9a0e75efb92c>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm_notebook(files,total=len(files)):


  0%|          | 0/212 [00:00<?, ?it/s]

In [405]:
def get_sample_urls(url,label):
    count=0
    files = sorted(glob('../../Gab_Data/new_features_old_gab/gab_fear_hate_features*.pickle'))
    for file in tqdm_notebook(files,total=len(files)):
            with open(file, 'rb') as handle:
                Gab_keyword_match = pickle.load(handle)

            if('predicted_probab' not in Gab_keyword_match[1].keys()):
                continue

            for element in Gab_keyword_match:
                try:
                    dt = datetime.fromisoformat(element['post_create_time'])
                    key_new=str(dt.month)+'/'+str(dt.year)
                except KeyError:
                    continue



                temp=element['post_body']
                all_urls = re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', temp)

                labels=[]

                for i in range(len(element['predicted_probab'])):
                    if(element['predicted_probab'][i]>threshold[reverse_dict_labels[i]]):
                        labels.append(reverse_dict_labels[i])

                    
                if(label in labels):
                    if(url in all_urls):
                        if(count>10):
                            break
                        print(element['post_body'])
                        count+=1
                        

In [387]:
dict_fear={}
dict_hate={}
dict_normal={}

for key in dict_urls.keys():
    for url_key in dict_urls[key].keys():
        try:
            dict_fear[key]+=dict_urls[key][url_key]['fearspeech']
        except KeyError:
            try:
                dict_fear[key]=dict_urls[key][url_key]['fearspeech']
            except KeyError:
                pass
        
        try:
            dict_hate[key]+=dict_urls[key][url_key]['hatespeech']
        except KeyError:
            try:
                dict_hate[key]=dict_urls[key][url_key]['hatespeech']
            except KeyError:
                pass
        
      

In [237]:
 with open('../Results/url_information.json', 'r') as openfile:
        # Reading from json file
        url_ground_truth = json.load(openfile)


In [377]:
domain_ground_truth={}
for key in url_ground_truth:
    domain_ground_truth[get_domain(key)]=url_ground_truth[key]

In [399]:
domain_ground_truth['easelzippers']

{'Factual Reporting': ' MIXED',
 'Country': ' USA',
 'World Press Freedom Rank': ' USA 45/180'}

In [ ]:
weaselzippers

In [390]:
top_fear_urls={k: v for k, v in sorted(dict_fear.items(), key=lambda item: item[1],reverse=True)[0:20]}
top_hate_urls={k: v for k, v in sorted(dict_hate.items(), key=lambda item: item[1],reverse=True)[0:20]}

In [396]:
top_fear_df=[]

for key in top_fear_urls.keys():
    print(key,dict_urls[key],top_fear_urls[key])
    print("================")

aclj {'http://www.aclj.org': {'fearspeech': 7}, 'http://aclj.us': {'fearspeech': 215}, 'http://aclj.org': {'fearspeech': 1}, 'https://aclj.us': {'fearspeech': 19}, 'https://aclj.org': {'fearspeech': 1}} 243
kek {'https://kek.gg': {'hatespeech': 1}, 'https://www.kek.gg': {'hatespeech': 1}, 'http://kek.gg': {'fearspeech': 159}} 159
islamexposedblog.blogspot {'http://islamexposedblog.blogspot.com': {'fearspeech': 70}, 'https://islamexposedblog.blogspot.co.uk': {'fearspeech': 2}} 72
dlvr {'http://dlvr.it': {'fearspeech': 67}, 'https://dlvr.it': {'fearspeech': 1}} 68
freecharitycars {'http://www.freecharitycars.org': {'fearspeech': 66}, 'http://freecharitycars.org': {'fearspeech': 2}} 68
nationaleconomicseditorial {'http://www.nationaleconomicseditorial.com': {'fearspeech': 47}, 'https://www.nationaleconomicseditorial.com': {'fearspeech': 7}, 'https://nationaleconomicseditorial.com': {'fearspeech': 4}} 58
hitenationnetwork {'http://www.whitenationnetwork.com': {'fearspeech': 54}} 54
1.cbn {

In [397]:

for key in top_hate_urls.keys():
    print(key,dict_urls[key],top_hate_urls[key])
    print("================")

pagesix {'https://pagesix.com': {'hatespeech': 65}, 'http://pagesix.com': {'hatespeech': 29}} 94
towleroad {'http://www.towleroad.com': {'hatespeech': 68}} 68
rodentguys {'https://www.rodentguys.com': {'hatespeech': 60}} 60
thewrap {'https://www.thewrap.com': {'hatespeech': 45}, 'http://www.thewrap.com': {'hatespeech': 11}} 56
easelzippers {'http://www.weaselzippers.us': {'hatespeech': 4}, 'https://www.weaselzippers.us': {'hatespeech': 42}} 46
pp.userapi {'https://pp.userapi.com': {'hatespeech': 36}} 36
bbs.dailystormer {'http://bbs.dailystormer.com': {'fearspeech': 1}, 'https://bbs.dailystormer.com': {'hatespeech': 32}, 'https://bbs.dailystormer.lol': {'hatespeech': 1}} 33
pornwikileaks {'http://www.pornwikileaks.com': {'hatespeech': 32}} 32
dailystormer {'http://www.dailystormer.com': {'fearspeech': 7}, 'https://www.dailystormer.com': {'fearspeech': 1}, 'https://dailystormer.ph': {'fearspeech': 1}, 'https://dailystormer.ai': {'hatespeech': 4}, 'https://dailystormer.red': {'hatespeech

In [408]:
get_sample_urls('https://pagesix.com','hatespeech')

<ipython-input-405-c4f3e90d1d1e>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm_notebook(files,total=len(files)):


  0%|          | 0/212 [00:00<?, ?it/s]

THIS DIRT BAG NEEDS TO BE PUBLICLY ____  ____  _____! 
(you should know those 3 words I cant say legally)
https://pagesix.com/2017/10/10/harvey-weinstein-accused-of-sexual-assault-in-another-expose/
BYE BYE Anti- Trump  scumbag https://pagesix.com/2017/10/10/harvey-weinstein-accused-of-sexual-assault-in-another-expose/
Because it's a Negro thing.

https://pagesix.com/2018/01/14/dennis-rodman-arrested-for-dui/
Negro rapper flaps his fat mouth once too often, once too much.

https://pagesix.com/2018/01/25/flavor-flav-beaten-up-in-las-vegas-casino/
ffs...

the nigger is a goddamn rapist.  I know, I know, he was just doing what a Nigger does according to his nature and he got away with it for years - lived long enough to enjoy his fame and his riches...SmDh

They say Old Nigger went berserk in the courtroom after  hearing his conviction.

https://pagesix.com/2018/04/27/cosbys-rep-compares-trial-to-emmett-till-lynching/
Leave it to insane liberals, and  soon we'll get a transvestite transge

In [395]:
dict_urls['nationaleconomicseditorial']

{'http://www.nationaleconomicseditorial.com': {'fearspeech': 47},
 'https://www.nationaleconomicseditorial.com': {'fearspeech': 7},
 'https://nationaleconomicseditorial.com': {'fearspeech': 4}}

### Archived data collection

In [25]:
import requests
import re
from bs4 import BeautifulSoup
import time


def parse_page(link):
    resp = requests.get(link)

    if resp.status_code == 404:
        return
    soup = BeautifulSoup(resp.content, 'html.parser')

    # get name
    site_name = soup.find_all("h1", class_="page-title")[0].text.strip(' \t\n\r')

    # get bias icon
    if site_name == "NDTV":
        bias_icon = "leftcenter06"
    elif site_name == "Knoxville News Sentinel":
        bias_icon = "leftcenter06"
    elif site_name == "Free Wheel Media":
        bias_icon = "leftcenter06"
    elif site_name == "Egyptian Streets":
        bias_icon = "leftcenter06"
    elif site_name == "The Times and Democrat":
        bias_icon = "leastbiased011"
    else:
        img = soup.find_all("div", class_="container mh-mobile")[0].find_all("img")[:1]
        src = img[0]["src"]
        pattern = re.compile("http.*\/([a-zA-z0-9]*)\.png")
        ms = pattern.match(src)
        bias_icon = ms.group(1)

    # get factual reporting rating
    factual_reporting_rating = None
    pgs = soup.find_all("p") 
    for pg in pgs:
        if ("Factual News:" in pg.text or "Factual Reporting:" in pg.text):
            try: 
                factual_reporting_rating = pg.find_all("strong")[0].text.strip('\n\r')
            except:
                try:
                    factual_reporting_rating = pg.find_all("b")[0].text.strip('\n\r')
                except:
                    factual_reporting_rating = pg.find_all("span")[0].text.strip('\n\r')


    if not factual_reporting_rating:
        divs = soup.find_all("div") 
        for div in divs:
            if ("Factual News:" in div.text or "Factual Reporting:" in div.text):
                try: 
                    factual_reporting_rating = div.find_all("strong")[0].text.strip('\n\r')
                except:
                    try:
                        factual_reporting_rating = div.find_all("b")[0].text.strip('\n\r')
                    except:
                        factual_reporting_rating = div.find_all("span")[0].text.strip('\n\r')

    if not factual_reporting_rating:
        raise Exception("no mas")




    # Get Domain
    if "Elko Daily Free Press" == site_name:
        domain="https://elkodaily.com/"
    elif "American Enterprise Institute" == site_name:
        domain="https://www.aei.org/"
    elif "Act.TV" == site_name:
        domain="http://act.tv"
    elif "The Fucking News" == site_name:
        domain="http://thefingnews.com/"
    elif "Kyiv Post" == site_name:
        domain="https://www.kyivpost.com/"
    elif "Philadelphia Tribune" == site_name:
        domain="http://www.phillytrib.com/"
    elif "Reuters" == site_name:
        domain="http://www.reuters.com/"
    elif "South Bend Tribune" == site_name:
        domain="https://www.southbendtribune.com/"
    elif "ConservativeOpinion.com" == site_name:
        domain="https://conservativeopinion.com/"
    else:    
        content = soup.find_all("div", class_="main")[0]
        anchors = content.find_all("a")
        for anchor in anchors:
            if (anchor.get("href", False) and 
               (site_name == "Wikipedia" or "wikipedia" not in anchor["href"]) and
               ("Sources:" in anchor.parent.text or "Notes:" in anchor.parent.text or "Source:" in anchor.parent.text)):

                domain = anchor["href"]

    return site_name, domain, bias_icon, factual_reporting_rating

urls = []
with open("all-links", "r") as f:
    with open("final-result", "a") as final_result:
        for link in f:
            res = parse_page(link.strip('\t\n\r'))
            if res:
                print(res)
                final_result.write(",".join(res) + "\n")


#link = 'https://mediabiasfactcheck.com/reuters/'
#print(parse_page(link))


FileNotFoundError: [Errno 2] No such file or directory: 'all-links'

In [133]:
urls=[
    'https://mediabiasfactcheck.com/pseudoscience-dictionary/',
    'https://mediabiasfactcheck.com/left/',
    'https://mediabiasfactcheck.com/center/',
    'https://mediabiasfactcheck.com/leftcenter/',
    'https://mediabiasfactcheck.com/right/',
    'https://mediabiasfactcheck.com/right-center/',
    'https://mediabiasfactcheck.com/conspiracy/',
    'https://mediabiasfactcheck.com/fake-news/',
    'https://mediabiasfactcheck.com/pro-science/',
    'https://mediabiasfactcheck.com/satire/'
]

In [134]:
import urllib
from bs4 import BeautifulSoup
from tqdm import tqdm,tqdm_notebook
all_links=[]

for url in tqdm_notebook(urls):
    text = requests.get(url)
    soup = BeautifulSoup(text.content,'html.parser')
    table=soup.find('div',{'id':'mh-wrapper'}).find('div',{'class':'mh-section mh-group'}).find('article').find('table')
    links = table.findAll('a')
    all_links+=links

<ipython-input-134-277904cc1df4>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for url in tqdm_notebook(urls):


  0%|          | 0/10 [00:00<?, ?it/s]

In [137]:
###
dict_urls={}

<a href="https://mediabiasfactcheck.com/pseudoscience-dictionary-5g-coronavirus-conspiracy/"><span style="font-size: 12pt;">5G-Coronavirus Conspiracy</span></a>

In [149]:
to_search_list=['Bias Rating',
 'Factual Reporting',
 'Country',
 'Press Freedom Rating',
 'Media Type',
 'Traffic/Popularity',
 'MBFC Credibility Rating']



In [211]:
# dict_urls={}


for link in tqdm_notebook(all_links[591:]):
    try:
        resp = requests.get(link['href'])
    except gaierror:
        continue
    soup = BeautifulSoup(resp.content, 'html.parser')
    try:
        paragraphs=soup.findAll('article')[0].findAll('p')
    except IndexError:
        continue
    dict_temp={}
    ## search for bias rating
    flag=0
    for para in paragraphs:
        for ele in to_search_list:
            if(ele in para.text):
                try:
                    lists=para.text.split('\n')
                    dict_temp={element.split(":")[0]:element.split(":")[1].replace(u'\xa0', u'') for element in lists}
                    flag+=1
                    break
                except IndexError:
                    pass
            if('Source' in para.text):
                try:
                    url=para.text.split('Source:')[1].replace(u'\xa0', u'')
                    flag+=1
                    break
                except IndexError:
                    pass

                
            
        if(flag==2):
            dict_urls[url]=dict_temp
            break
        

<ipython-input-211-2c545c83f228>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for link in tqdm_notebook(all_links[591:]):


  0%|          | 0/3150 [00:00<?, ?it/s]

In [213]:
list(dict_urls.keys())[0]

' https://www.accountable.us/'

In [235]:
with open('../Results/url_information.json', 'w') as fp:
    json.dump(dict_urls, fp,indent=4)


In [142]:
soup.findAll('article')[0].findAll('p')[8].text.split('\n')

['Source: https://speld.nl/']

In [153]:
dict_urls

{'https://mediabiasfactcheck.com/pseudoscience-dictionary-5g-coronavirus-conspiracy/': {},
 'https://mediabiasfactcheck.com/pseudoscience-dictionary-acupressure/': {},
 'https://mediabiasfactcheck.com/pseudoscience-directory-acupuncture/': {},
 'https://mediabiasfactcheck.com/pseudoscience-dictionary-alternative-cancer-treatments/': {},
 'https://mediabiasfactcheck.com/pseudoscience-dictionary-alternative-medicine/': {},
 'https://mediabiasfactcheck.com/pseudoscience-dictionary-aids-denialism/': {},
 'https://mediabiasfactcheck.com/pseudoscience-dictionary-ancient-astronauts/': {},
 'https://mediabiasfactcheck.com/pseudoscience-dictionary-anti-vaccination-anti-vax/': {},
 'https://mediabiasfactcheck.com/pseudoscience-dictionary-applied-kinesiology/': {},
 'https://mediabiasfactcheck.com/pseudoscience-dictionary-aroma-therapy/': {}}

In [154]:
dict_temp

{}

In [157]:
to_search_list

['Bias Rating',
 'Factual Reporting',
 'Country',
 'Press Freedom Rating',
 'Media Type',
 'Traffic/Popularity',
 'MBFC Credibility Rating']